# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [41]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [42]:
weather_df = pd.read_csv("weather_export.csv")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hemisphere
0,faanui,-16.48,-151.75,80.08,72,4,11.39,PF,1607898555,Southern
1,erzin,36.96,36.20,59.00,65,100,5.01,TR,1607898555,Northern
2,montrose,38.42,-108.27,25.00,57,31,3.00,US,1607898556,Northern
3,thompson,55.74,-97.86,-14.80,75,20,5.82,CA,1607898556,Northern
4,bahia blanca,-38.72,-62.27,73.81,21,6,7.78,AR,1607898552,Southern


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [43]:
gmaps.configure(api_key=g_key)

In [44]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
* Drop any rows will null values.

In [45]:
good_weather_df = weather_df[(weather_df["Max Temp"] > 70) \
                             & (weather_df["Max Temp"] < 80) \
                             & (weather_df["Wind Speed"] > 10) \
                             & (weather_df["Cloudiness"] == 0)]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
hotel_df = good_weather_df[["City", "Country", "Lat", "Lng"]].copy()

hotel_df["Hotel Name"] = "" 

hotel_df

,City,Country,Lat,Lng,Hotel Name
97,dwarka,IN,22.24,68.97,
141,beloha,MG,-25.17,45.05,
203,morrope,PE,-6.54,-80.01,
288,ormara,PK,25.21,64.64,
301,vila velha,BR,-20.33,-40.29,
442,guerrero negro,MX,27.98,-114.06,


In [53]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
}

hotel_df["Hotel Lat"] = "" 
hotel_df["Hotel Lng"] = ""

for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lng = row["Lng"]

    # add keyword to params dict
    params['location'] = f"{lat}, {lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df["Hotel Name"][index] = results[0]['name']
        hotel_df["Hotel Lat"][index] = float(results[0]['geometry']['location']['lat'])
        hotel_df["Hotel Lng"][index] = float(results[0]['geometry']['location']['lng'])
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
hotel_df

Retrieving Results for Index 97: dwarka.
Closest hotel is Hotel Rajmandir.
------------
Retrieving Results for Index 141: beloha.


c:\users\mary\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\mary\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\mary\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Missing field/result... skipping.
------------
Retrieving Results for Index 203: morrope.
Closest hotel is Hospedaje Santa Rosa.
------------
Retrieving Results for Index 288: ormara.
Closest hotel is Alqadri hotel zero point ormara.
------------
Retrieving Results for Index 301: vila velha.
Closest hotel is Hotel Prainha.
------------
Retrieving Results for Index 442: guerrero negro.
Closest hotel is Malarrimo.
------------


,City,Country,Lat,Lng,Hotel Name,Hotel Lat,Hotel Lng
97,dwarka,IN,22.24,68.97,Hotel Rajmandir,22.2394,68.9696
141,beloha,MG,-25.17,45.05,Beloha Health Centre,,
203,morrope,PE,-6.54,-80.01,Hospedaje Santa Rosa,-6.54011,-80.0121
288,ormara,PK,25.21,64.64,Alqadri hotel zero point ormara,25.2493,64.5967
301,vila velha,BR,-20.33,-40.29,Hotel Prainha,-20.3311,-40.2918
442,guerrero negro,MX,27.98,-114.06,Malarrimo,27.9685,-114.05


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [55]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Hotel Rajmandir</dd>\n<dt>City</dt><dd>dwarka</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Beloha Health Centre</dd>\n<dt>City</dt><dd>beloha</dd>\n<dt>Country</dt><dd>MG</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hospedaje Santa Rosa</dd>\n<dt>City</dt><dd>morrope</dd>\n<dt>Country</dt><dd>PE</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Alqadri hotel zero point ormara</dd>\n<dt>City</dt><dd>ormara</dd>\n<dt>Country</dt><dd>PK</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Prainha</dd>\n<dt>City</dt><dd>vila velha</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Malarrimo</dd>\n<dt>City</dt><dd>guerrero negro</dd>\n<dt>Country</dt><dd>MX</dd>\n</dl>\n']

In [57]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel_df[["Lat", "Lng"]], info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))